In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
major_df = pd.read_csv('major.csv')


X = train_df.review
Y = train_df.category

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)





AttributeError: 'DataFrame' object has no attribute 'category'

In [28]:
# Create a pipeline with CountVectorizer and Multinomial Naive Bayes classifier

model = make_pipeline(CountVectorizer(), MultinomialNB())


model.fit(X_train,Y_train)

predictions = model.predict(X_test)
accuracy = accuracy_score(Y_test, predictions)
accuracy



0.8855633802816901